In [57]:
import pandas as pd
import numpy as np
import openpyxl
import pyreadr
import seaborn as sns
import matplotlib.pyplot as plt
import string
import torch
import torch.nn as nn
from torch.nn import functional as F


# Load Jacobson Data

- Import Jacobson data from 1946 onwards and convert to workable csv file.
- Print the first five rows for inspection.

In [58]:
result = pyreadr.read_r('house_election_data_46_22.rds')
df = result[None]

print(df.head())
df.to_csv('all_data.csv', index=False)

     year   stcd  inc  pwin           dv          dvp   fr  po1  po2  redist  \
0  1946.0  101.0  1.0   1.0 -999999999.0 -999999999.0  0.0  5.0  2.0     0.0   
1  1946.0  102.0  1.0   1.0 -999999999.0 -999999999.0  0.0  5.0  2.0     0.0   
2  1946.0  103.0  1.0   1.0 -999999999.0 -999999999.0  0.0  5.0  2.0     0.0   
3  1946.0  104.0  1.0   1.0         88.1         84.5  0.0  0.0  0.0     0.0   
4  1946.0  105.0  1.0   1.0 -999999999.0 -999999999.0  1.0  5.0  2.0     0.0   

          dexp         rexp        dpres  
0 -999999999.0 -999999999.0 -999999999.0  
1 -999999999.0 -999999999.0 -999999999.0  
2 -999999999.0 -999999999.0 -999999999.0  
3 -999999999.0 -999999999.0 -999999999.0  
4 -999999999.0 -999999999.0 -999999999.0  


# Convert Data Types
- All non-voteshare columns are either factor variables or campaign spending and can be converted to integers.

In [59]:
# Convert all columns to integers except for the exclude_columns

exclude_columns = ["dv", "dvp", "dpres"]

for column in df.columns:
    if column not in exclude_columns:
        df[column] = df[column].astype(float).fillna(0).astype(int)

print(df.head())

   year  stcd  inc  pwin           dv          dvp  fr  po1  po2  redist  \
0  1946   101    1     1 -999999999.0 -999999999.0   0    5    2       0   
1  1946   102    1     1 -999999999.0 -999999999.0   0    5    2       0   
2  1946   103    1     1 -999999999.0 -999999999.0   0    5    2       0   
3  1946   104    1     1         88.1         84.5   0    0    0       0   
4  1946   105    1     1 -999999999.0 -999999999.0   1    5    2       0   

        dexp       rexp        dpres  
0 -999999999 -999999999 -999999999.0  
1 -999999999 -999999999 -999999999.0  
2 -999999999 -999999999 -999999999.0  
3 -999999999 -999999999 -999999999.0  
4 -999999999 -999999999 -999999999.0  


# Fill in NaN Values
- Negative entries in Jacobson are placeholders for missing data.

In [60]:
# Turn negative values into NaNs

df = df.mask(df < 0, np.nan)
print(df.head())

   year  stcd  inc  pwin    dv   dvp  fr  po1  po2  redist  dexp  rexp  dpres
0  1946   101    1   1.0   NaN   NaN   0  5.0  2.0       0   NaN   NaN    NaN
1  1946   102    1   1.0   NaN   NaN   0  5.0  2.0       0   NaN   NaN    NaN
2  1946   103    1   1.0   NaN   NaN   0  5.0  2.0       0   NaN   NaN    NaN
3  1946   104    1   1.0  88.1  84.5   0  0.0  0.0       0   NaN   NaN    NaN
4  1946   105    1   1.0   NaN   NaN   1  5.0  2.0       0   NaN   NaN    NaN


# Split "stcd" into "state" and "dist"

In [61]:
# Split "stcd" into state and district
def split_stcd(stcd):
    stcd_str = str(stcd)
    if len(stcd_str) == 3:
        state = int(stcd_str[0])
        dist = int(stcd_str[1:])
    elif len(stcd_str) == 4:
        state = int(stcd_str[:2])
        dist = int(stcd_str[2:])
    else:
        raise ValueError("Invalid length of stcd")
    return state, dist

# Apply the function to split the stcd column
df[['state', 'dist']] = df['stcd'].apply(split_stcd).apply(pd.Series)

# Create Categorical Factor Mappings
- Continuous values must be converted to factors, and so must be binned reasonably
- map_exp creates expenditure categories ranging from $\le \$50,000$ to $> \$25,000,000$.
- map_voteshare creates voteshare categories ranging from $\le 10\%$ to $>90\%$.
- Both categories become more granular (smaller bin sizes) around the mode of each category.
- Apply these mappings to all applicable columns and delete remaining extraneous columns.

## Additionally Create Broader Categories
- Since multicolinearity is not a concern, adding less granular categories will not detract from model accuracy
- We can theoretically consider as many different sized bins as we like

In [62]:
# Turn real variables into ranges using maps
def map_exp_granular(value):
    if value <= 50000:
        return 0
    elif value <= 100000:
        return 1
    elif value <= 200000:
        return 2
    elif value <= 300000:
        return 3
    elif value <= 400000:
        return 4
    elif value <= 500000:
        return 5
    elif value <= 600000:
        return 6
    elif value <= 700000:
        return 7
    elif value <= 800000:
        return 8
    elif value <= 900000:
        return 9
    elif value <= 1000000:
        return 10
    elif value <= 1250000:
        return 11
    elif value <= 1500000:
        return 12
    elif value <= 1750000:
        return 13
    elif value <= 2000000:
        return 14
    elif value <= 2500000:
        return 15
    elif value <= 3000000:
        return 16
    elif value <= 4000000:
        return 17
    elif value <= 5000000:
        return 18
    elif value <= 6000000:
        return 19
    elif value <= 7500000:
        return 20
    elif value <= 9000000:
        return 21
    elif value <= 12000000:
        return 22
    elif value <= 15000000:
        return 23
    elif value <= 17500000:
        return 24
    elif value <= 20000000:
        return 25
    elif value <= 22500000:
        return 26
    elif value <= 25000000:
        return 27
    else: pass

def map_exp(value):
    if value <= 100000:
        return 0
    elif value <= 400000:
        return 1
    elif value <= 600000:
        return 2
    elif value <= 900000:
        return 3
    elif value <= 1250000:
        return 4
    elif value <= 1500000:
        return 5
    elif value <= 2000000:
        return 6
    elif value <= 3000000:
        return 7
    elif value <= 6000000:
        return 8
    elif value <= 9000000:
        return 9
    elif value <= 12000000:
        return 10
    elif value <= 17500000:
        return 11
    elif value <= 22500000:
        return 12
    else: pass

df['dexp_cat_gran'] = df['dexp'].apply(map_exp_granular)
df['rexp_cat_gran'] = df['rexp'].apply(map_exp_granular)

df['dexp_cat'] = df['dexp'].apply(map_exp)
df['rexp_cat'] = df['rexp'].apply(map_exp)

def map_voteshare(value):
    if value <= 10:
        return 0
    elif value <= 15:
        return 1
    elif value <= 20:
        return 2
    elif value <= 25:
        return 3
    elif value <= 30:
        return 4
    elif value <= 33:
        return 5
    elif value <= 36:
        return 6
    elif value <= 39:
        return 7
    elif value <= 41:
        return 8
    elif value <= 43:
        return 9
    elif value <= 45:
        return 10
    elif value <= 46:
        return 11
    elif value <= 47:
        return 12
    elif value <= 48:
        return 13
    elif value <= 49:
        return 14
    elif value <= 49.5:
        return 15
    elif value <= 50:
        return 16
    elif value <= 50.5:
        return 17
    elif value <= 51:
        return 18
    elif value <= 52:
        return 19
    elif value <= 53:
        return 20
    elif value <= 54:
        return 21
    elif value <= 55:
        return 22
    elif value <= 57:
        return 23
    elif value <= 59:
        return 24
    elif value <= 61:
        return 25
    elif value <= 64:
        return 26
    elif value <= 67:
        return 27
    elif value <= 70:
        return 28
    elif value <= 75:
        return 29
    elif value <= 80:
        return 30
    elif value <= 85:
        return 31
    elif value <= 90:
        return 32
    elif value <= 100:
        return 33
    else: pass

df['dpres_cat'] = df['dpres'].apply(map_voteshare)
df['dvp_cat'] = df['dvp'].apply(map_voteshare)
df['dv_cat'] = df['dv'].apply(map_voteshare)

# Delete original columns
del df['dvp'], df['dpres'], df['dexp'], df['rexp'], df['stcd'], df['dv']

print(df.head())


   year  inc  pwin  fr  po1  po2  redist  state  dist  dexp_cat_gran  \
0  1946    1   1.0   0  5.0  2.0       0      1     1            NaN   
1  1946    1   1.0   0  5.0  2.0       0      1     2            NaN   
2  1946    1   1.0   0  5.0  2.0       0      1     3            NaN   
3  1946    1   1.0   0  0.0  0.0       0      1     4            NaN   
4  1946    1   1.0   1  5.0  2.0       0      1     5            NaN   

   rexp_cat_gran  dexp_cat  rexp_cat  dpres_cat  dvp_cat  dv_cat  
0            NaN       NaN       NaN        NaN      NaN     NaN  
1            NaN       NaN       NaN        NaN      NaN     NaN  
2            NaN       NaN       NaN        NaN      NaN     NaN  
3            NaN       NaN       NaN        NaN     31.0    32.0  
4            NaN       NaN       NaN        NaN      NaN     NaN  


# Create Seperate File for Output Variables
- In this case, 'pwin' and 'dv_cat.'

In [63]:
# Filter and create target df
filt = df.drop(columns = ["pwin", "dv_cat"])
out = df[["pwin", "dv_cat"]]

out.loc[:, 'pwin'] = out['pwin'].astype('Int64')

print(filt.head())
print(out.head())

   year  inc  fr  po1  po2  redist  state  dist  dexp_cat_gran  rexp_cat_gran  \
0  1946    1   0  5.0  2.0       0      1     1            NaN            NaN   
1  1946    1   0  5.0  2.0       0      1     2            NaN            NaN   
2  1946    1   0  5.0  2.0       0      1     3            NaN            NaN   
3  1946    1   0  0.0  0.0       0      1     4            NaN            NaN   
4  1946    1   1  5.0  2.0       0      1     5            NaN            NaN   

   dexp_cat  rexp_cat  dpres_cat  dvp_cat  
0       NaN       NaN        NaN      NaN  
1       NaN       NaN        NaN      NaN  
2       NaN       NaN        NaN      NaN  
3       NaN       NaN        NaN     31.0  
4       NaN       NaN        NaN      NaN  
   pwin  dv_cat
0     1     NaN
1     1     NaN
2     1     NaN
3     1    32.0
4     1     NaN


# What to do With Missing Data?
1. **Nothing**: Machine learning algorithm in pred2022.ipynb is capable of only training givin only the available data
2. **Missing as a Vocab Item**: Allow the model to treat missingness as a datapoint, by storing any NaN as a vocabulary item
3. **Impute with Mean**: Impute any missing item with the average value of the column or year
4. **Impute with Mode**: Impute any missing item with the most common value of the column or year

In [64]:
###################################################################
# Strategy 2: This code converts all missing data to a unique value to be turned into a vocab item

## The value (2342 here) just needs to be unique to that column and carries no real value

### We cannot do this to missing outcomes, we do not want the model predicting NaN results
###################################################################

filt_nona = filt.fillna(2342)
print(filt_nona.head())
print(out.head())

# I want to export this as my data, so for now I will call this 'filt' for the code below
filt = filt_nona

   year  inc  fr  po1  po2  redist  state  dist  dexp_cat_gran  rexp_cat_gran  \
0  1946    1   0  5.0  2.0       0      1     1         2342.0         2342.0   
1  1946    1   0  5.0  2.0       0      1     2         2342.0         2342.0   
2  1946    1   0  5.0  2.0       0      1     3         2342.0         2342.0   
3  1946    1   0  0.0  0.0       0      1     4         2342.0         2342.0   
4  1946    1   1  5.0  2.0       0      1     5         2342.0         2342.0   

   dexp_cat  rexp_cat  dpres_cat  dvp_cat  
0    2342.0    2342.0     2342.0   2342.0  
1    2342.0    2342.0     2342.0   2342.0  
2    2342.0    2342.0     2342.0   2342.0  
3    2342.0    2342.0     2342.0     31.0  
4    2342.0    2342.0     2342.0   2342.0  
   pwin  dv_cat
0     1     NaN
1     1     NaN
2     1     NaN
3     1    32.0
4     1     NaN


# Housekeeping
- Check how many NaNs are in the dataset. If we have executed cell 28, this will be 0 since NaNs have been treated as arbitrary numbers.
- Make sure all values are integers if they are supposed to be.
- Print the number of unique entries of each column to get a sense of vocabulary size.

In [65]:
filt.isnull().sum()
out.isnull().sum()

# Turn columns of filt into integers if they are float
for col in filt.columns:
    filt.loc[:, col] = filt[col].astype('Int64')

for col in out.columns:
    out.loc[:, col] = out[col].astype('Int64')

# Print the number of unique items in each column to get a sense of "vocabulary" size
for column in filt:
    try: 
        unique_vals = np.unique(df[column])
    except:
        unique_vals = df[column].unique()

    nr_vals = len(unique_vals)
    if nr_vals < 10:
        print('The number of values for feature {} :{} -- {}'.format(column, nr_vals, unique_vals))
    else:
        print('The number of values for feature {}:{}'.format(column, nr_vals))

The number of values for feature year:39
The number of values for feature inc:12
The number of values for feature fr :6 -- [0 1 2 3 8 9]
The number of values for feature po1:10
The number of values for feature po2:11
The number of values for feature redist :4 -- [0 1 4 6]
The number of values for feature state:50
The number of values for feature dist:53
The number of values for feature dexp_cat_gran:28
The number of values for feature rexp_cat_gran:28
The number of values for feature dexp_cat:14
The number of values for feature rexp_cat:14
The number of values for feature dpres_cat:35
The number of values for feature dvp_cat:35


# Plot Data
- If desired, examine relationships between any two variables.

In [66]:
# g = sns.pairplot(df[['inc', 'dvp_cat', 'po1', 'rexp_cat', 'dexp_cat', 'state', 'dist', 'pwin', 'dpres_cat']], hue='pwin')

# Add Column IDs
- A number of columns contain the same items, but the neural network must still be able to distinguish between them. Since column order may be shuffled in the network, add column identification letters to each non-NaN element of the dataset.
- This creates a full vocabulary of inputs and outputs representing all entries of each column that can be utilized by the neural network eventually.

In [67]:
# Map each column to a letter
column_mapping_filt = dict(zip(filt.columns, string.ascii_lowercase))
column_mapping_out = dict(zip(out.columns, string.ascii_lowercase))

def append_letter(val, col_name, column_mapping):
    if pd.notna(val):
        return f"{column_mapping[col_name]}{val}"
    else:
        return val  # Return NaN if the value is NaN

# Apply the function element-wise to the DataFrame
for col in filt.columns:
    filt.loc[:, col] = filt[col].apply(lambda x, col_name=col: append_letter(x, col_name, column_mapping_filt))

# Apply the function element-wise to outputs using .loc as well
for col in out.columns:
    out.loc[:, col] = out[col].apply(lambda x, col_name=col: append_letter(x, col_name, column_mapping_out))

# Convert all values in the DataFrame to strings
filt = filt.astype(str)
filt = filt.replace("<NA>", pd.NA)

out = out.astype(str)
out = out.replace("<NA>", pd.NA)

print(filt.head())
print(out.head)

    year inc  fr   po1   po2 redist state dist dexp_cat_gran rexp_cat_gran  \
0  a1946  b1  c0  d5.0  e2.0     f0    g1   h1       i2342.0       j2342.0   
1  a1946  b1  c0  d5.0  e2.0     f0    g1   h2       i2342.0       j2342.0   
2  a1946  b1  c0  d5.0  e2.0     f0    g1   h3       i2342.0       j2342.0   
3  a1946  b1  c0  d0.0  e0.0     f0    g1   h4       i2342.0       j2342.0   
4  a1946  b1  c1  d5.0  e2.0     f0    g1   h5       i2342.0       j2342.0   

  dexp_cat rexp_cat dpres_cat  dvp_cat  
0  k2342.0  l2342.0   m2342.0  n2342.0  
1  k2342.0  l2342.0   m2342.0  n2342.0  
2  k2342.0  l2342.0   m2342.0  n2342.0  
3  k2342.0  l2342.0   m2342.0    n31.0  
4  k2342.0  l2342.0   m2342.0  n2342.0  
<bound method NDFrame.head of       pwin dv_cat
0       a1   <NA>
1       a1   <NA>
2       a1   <NA>
3       a1    b32
4       a1   <NA>
...    ...    ...
16963   a0     b6
16964   a0   <NA>
16965   a0     b7
16966   a0   <NA>
16967   a0     b4

[16968 rows x 2 columns]>


# Convert to CSV

In [33]:
filt.to_csv('filtered_data.csv', index=False)
out.to_csv('targets.csv', index=False)